# First I download the necessary libraries

In [1]:

import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import matplotlib.cm as cm
import matplotlib.colors as colors

! pip install folium==0.5.0
import folium # plotting library

print('\n Libraries Downloaded!')

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes

 Libraries Downloaded!


### Next, I set up to make the soup

In [2]:
# First, the parameters 
url = 'https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=1011037969'
html=requests.get(url).text
soup = BeautifulSoup(html, 'html.parser')
locgeo_df = pd.read_csv('https://cocl.us/Geospatial_data', index_col='Postal Code')
print("Parameters set")

Parameters set


### Extracting the table from the site

In [3]:
table = soup.findAll('table')
trs = table[0].findAll('tr')

# helper method to extracting all information from a list of 'tr' tag inside table
def find_all_element(trs):
    result = []
    for tr in trs:
        td = tr.findAll('td')
        # if tr contain td tags other than 3, continue
        if len(td) !=3:
            continue
        postcode = td[0].text.replace('\n','')
        borough = td[1].text.replace('\n','')
        
        # only processing data other than 'Not assigned'
        if borough == 'Not assigned':
            continue
        neigh = td[2].text.replace('\n','')
        if neigh == 'Not assigned':
            neigh = borough
        geo_coord = locgeo_df[locgeo_df.index==postcode]
        
        ## get coordinate
        l1 = np.float(geo_coord['Latitude'])
        l2 = np.float(geo_coord['Longitude'])
        buffer_dict = {'POST_CODE':postcode,'BOROUGH':borough, 'neighbour':neigh.strip(),'Latitude':l1,'Longitude':l2}
        result.append(buffer_dict)
    return pd.DataFrame(result)  

df = find_all_element(trs)
df

,POST_CODE,BOROUGH,neighbour,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


# Now begins the exploration

### Making the map with the wiki data

In [6]:
# I optain the coordinates from Google
latitude = 43.65426
longitude = -79.36063

# Setting the map
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['BOROUGH'], df['neighbour']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Linking Foursquare

In [7]:
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value
CLIENT_ID = 'IQQVXNPOHGE1LIMOIPA1APTTVNRADRVZRCVYTYEZXS0XPYEI' # your Foursquare ID
CLIENT_SECRET = 'VX0U2Z1XMGFWX3LVBTTETKA3SY4GBYOIHBVMZAC01HPU44GO' # your Foursquare Secret
ACCESS_TOKEN = 'M1QJCBOILTSJBJ5GIHQZOKR2ZC20UXIIHY34JJI5YSSIFEL4' # your FourSquare Access Token
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: IQQVXNPOHGE1LIMOIPA1APTTVNRADRVZRCVYTYEZXS0XPYEI
CLIENT_SECRET:VX0U2Z1XMGFWX3LVBTTETKA3SY4GBYOIHBVMZAC01HPU44GO


### Next, I set up the function to map the Foursquare Data

In [8]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Next, I make a table with the venues

In [10]:
print('Toronto venues shape: ', toronto_venues.shape)
toronto_venues.head(10)

Toronto venues shape:  (2118, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
5,Victoria Village,43.725882,-79.315572,Pizza Nova,43.725824,-79.312860,Pizza Place
6,"Regent Park, Harbourfront",43.654260,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
7,"Regent Park, Harbourfront",43.654260,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
8,"Regent Park, Harbourfront",43.654260,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
9,"Regent Park, Harbourfront",43.654260,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


In [11]:
toronto_venues.groupby('Venue Category').count()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
Accessories Store,3,3,3,3,3,3
Adult Boutique,1,1,1,1,1,1
Airport,2,2,2,2,2,2
Airport Food Court,1,1,1,1,1,1
Airport Lounge,2,2,2,2,2,2
...,...,...,...,...,...,...
Warehouse Store,1,1,1,1,1,1
Wine Bar,9,9,9,9,9,9
Wings Joint,1,1,1,1,1,1


In [12]:
toronto_venues.groupby('Venue').count()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue Latitude,Venue Longitude,Venue Category
Venue,,,,,,
(The New) Moksha Yoga Bloor West,1,1,1,1,1,1
195 Jane Rocket,1,1,1,1,1,1
2 Bros Cuisine,1,1,1,1,1,1
306 Yonge Street - Jordan Store,1,1,1,1,1,1
360 Restaurant,1,1,1,1,1,1
...,...,...,...,...,...,...
lululemon,1,1,1,1,1,1
lululemon athletica,2,2,2,2,2,2
nodo,1,1,1,1,1,1


In [16]:
toronto_venues.groupby('Venue').mean()

,Neighborhood Latitude,Neighborhood Longitude,Venue Latitude,Venue Longitude
Venue,,,,
(The New) Moksha Yoga Bloor West,43.651571,-79.484450,43.648658,-79.485242
195 Jane Rocket,43.673185,-79.487262,43.672335,-79.492634
2 Bros Cuisine,43.696319,-79.532242,43.692499,-79.531698
306 Yonge Street - Jordan Store,43.657162,-79.378937,43.656495,-79.381015
360 Restaurant,43.640816,-79.381752,43.642537,-79.387042
...,...,...,...,...
lululemon,43.715383,-79.405678,43.713478,-79.400082
lululemon athletica,43.654328,-79.377177,43.653286,-79.380764
nodo,43.661608,-79.464763,43.665303,-79.465621
